In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import os
import datetime
import tensorflow_hub as hub
import numpy as np
    
import pandas as pd
from sklearn.model_selection import train_test_split

### Data taken from CFPB, the data when a consumer files a grievance against a bank. The target variable is the product column here.

In [3]:
df=pd.read_csv('https://github.com/srivatsan88/YouTubeLI/blob/master/dataset/consumer_compliants.zip?raw=true', compression='zip', sep=',', quotechar='"')

In [4]:
df.columns

Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Consumer disputed?', 'Complaint ID'],
      dtype='object')

In [5]:
pd.set_option('display.max_colwidth', -1)
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,4/3/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Fraudulent loan,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account.",Company has responded to the consumer and the CFPB and chooses not to provide a public response,TRUIST FINANCIAL CORPORATION,PA,None,None,Consent provided,Web,4/3/2020,Closed with explanation,Yes,NaN,3591341
1,3/12/2020,Debt collection,Payday loan debt,Attempts to collect debt not owed,Debt is not yours,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back.",None,CURO Intermediate Holdings,CO,806XX,None,Consent provided,Web,3/12/2020,Closed with explanation,Yes,NaN,3564184
2,2/6/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Credit denial,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times.",None,CAPITAL ONE FINANCIAL CORPORATION,OH,430XX,None,Consent provided,Web,2/6/2020,Closed with explanation,Yes,NaN,3521949
3,3/6/2020,Checking or savings account,Savings account,Managing an account,Banking errors,"Please see CFPB case XXXX. \n\nCapital One, in the letter they provided ( and attached to that case as their response ) said this : "" The funds were reversed and sent back to XXXX XXXX XXXX on XX/XX/XXXX ''. \n\nXXXX XXXX XXXX ( now XXXX XXXX ) has not received these funds. Staff at XXXX XXXX - and also staff at the account-holder 's business - have looked for return of my money ( {$650.00} ) and find nothing. \n\nCapital One needs to document - actually prove - they returned the funds, as stated in their letter. Capital One must provide electronic information, if the return was made that way, or document the paper check they sent back to XXXX XXXX. \n\nI've left 3 messages about this problem for the person who signed the letter 

In [6]:
X_train, X_test = train_test_split(df, test_size=0.2, random_state=42)

In [7]:
df['Product'].value_counts()

Debt collection                21772
Credit card or prepaid card    13193
Mortgage                       9799 
Checking or savings account    7003 
Student loan                   2950 
Vehicle loan or lease          2736 
Name: Product, dtype: int64

### We can see here that the first 2 classes have high numbers but the others are not too representative. Implying the dataset is unbalanced.

In [8]:
from sklearn.utils import class_weight
class_weights = list(class_weight.compute_class_weight(
    class_weight = 'balanced',
    classes = np.unique(df['Product']),
    y = df['Product']))  
#here we assign a higher weight to a class that is less represented and lower weight to a class that is more

In [9]:
class_weights.sort()

In [10]:
class_weights

[0.43980801028844385,
 0.7258015614340938,
 0.9771915501581794,
 1.3673425674710837,
 3.2459322033898306,
 3.4998172514619883]

In [11]:
weights={}
for index, weight in enumerate(class_weights) :
    weights[index]=weight

In [12]:
weights  #as expected the weigts for the frequent classes are less

{0: 0.43980801028844385,
 1: 0.7258015614340938,
 2: 0.9771915501581794,
 3: 1.3673425674710837,
 4: 3.2459322033898306,
 5: 3.4998172514619883}

### We would use the 'Product' column as the target and the 'Consumer complaint narrative' as the feature text

In [13]:
dataset_train = tf.data.Dataset.from_tensor_slices((X_train['Consumer complaint narrative'].values, X_train['Product'].values))
dataset_test = tf.data.Dataset.from_tensor_slices((X_test['Consumer complaint narrative'].values, X_test['Product'].values))

In [14]:
for text, target in dataset_train.take(5):
    print('Complaint: {}, Target: {}'.format(text, target))

Complaint: b'I am a victim of identity theft and this debt does not belong to me. \n\nPlease see the identity theft report and legal affidavit attached.', Target: b'Debt collection'
Complaint: b'I signed up for Citibank Checking and Savings account promotion program. Nowhere in their promotion requirements does it say that it does not apply to Texas. You can see their promotion published here, all it said was that " To enroll in this offer you must apply directly from this page. Citi rates and promotional offers may vary by state. Looks like you\'re in Texas. \'\' I even re-selected the state to make sure it does not tell me that Texas is not eligible, but it didn\'t say anything. Also in their fine prints they did not list the states that were eligible so who would know if they don\'t qualify? This is scam by misleading people in believing they will would qualify so they will open an account with them. See the advertised deal here, you can see nothing about the states. \n\nhttps : //b

In [15]:
for text, target in dataset_test.take(5):
    print('Complaint: {}, Target: {}'.format(text, target))

Complaint: b"On XX/XX/XXXX, I paid off the account in full to the original creditor. During this phone call, the original creditor informed me that they will close the account with the debt collection agency. \n\nOn XX/XX/XXXX, I called the original creditor regarding the account closure because I will still seeing the debt on my credit report. During this phone call, they said they had emailed the supervisor of the collection agency on XX/XX/XXXX to close the account and that the collection account will be closed soon. \n\nOn XX/XX/XXXX, I called the supervisor of the collection agency who informed me that he never received any communication from the original creditor regarding the closure of this account. He informed me that he needed a formal confirmation from the original creditor in order to close the account. I placed the supervisor of the collection agency on a 3-way phone call with the original creditor and myself. During this phone call, the supervisor told the original credit

### Converting the target to numerical representation of classes

In [16]:
table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant(['Debt collection', 'Credit card or prepaid card', 'Mortgage', 'Checking or savings account', 'Student loan', 'Vehicle loan or lease']),
        values=tf.constant([0, 1, 2, 3, 4, 5]),
    ),
    default_value=tf.constant(-1),
    name="target_encoding"
)

@tf.function
def target(x):
    return table.lookup(x)

In [17]:
#printing the feateure text and converted target
def show_batch(dataset, size=5):
    for batch, label in dataset.take(size):
        print(batch.numpy())
        print(target(label).numpy())

In [18]:
show_batch(dataset_test,6)

b"On XX/XX/XXXX, I paid off the account in full to the original creditor. During this phone call, the original creditor informed me that they will close the account with the debt collection agency. \n\nOn XX/XX/XXXX, I called the original creditor regarding the account closure because I will still seeing the debt on my credit report. During this phone call, they said they had emailed the supervisor of the collection agency on XX/XX/XXXX to close the account and that the collection account will be closed soon. \n\nOn XX/XX/XXXX, I called the supervisor of the collection agency who informed me that he never received any communication from the original creditor regarding the closure of this account. He informed me that he needed a formal confirmation from the original creditor in order to close the account. I placed the supervisor of the collection agency on a 3-way phone call with the original creditor and myself. During this phone call, the supervisor told the original creditor that no 

In [19]:
#function to apply the one hot encoding to the classes
def fetch(text, labels):
    return text, tf.one_hot(target(labels),6)

In [34]:
train_data_f=dataset_train.map(fetch)
test_data_f=dataset_test.map(fetch)

In [35]:
next(iter(train_data_f))

(<tf.Tensor: shape=(), dtype=string, numpy=b'I am a victim of identity theft and this debt does not belong to me. \n\nPlease see the identity theft report and legal affidavit attached.'>,
 <tf.Tensor: shape=(6,), dtype=float32, numpy=array([1., 0., 0., 0., 0., 0.], dtype=float32)>)

In [36]:
train_data, train_labels = next(iter(train_data_f.batch(5)))
train_data, train_labels

(<tf.Tensor: shape=(5,), dtype=string, numpy=
 array([b'I am a victim of identity theft and this debt does not belong to me. \n\nPlease see the identity theft report and legal affidavit attached.',
        b'I signed up for Citibank Checking and Savings account promotion program. Nowhere in their promotion requirements does it say that it does not apply to Texas. You can see their promotion published here, all it said was that " To enroll in this offer you must apply directly from this page. Citi rates and promotional offers may vary by state. Looks like you\'re in Texas. \'\' I even re-selected the state to make sure it does not tell me that Texas is not eligible, but it didn\'t say anything. Also in their fine prints they did not list the states that were eligible so who would know if they don\'t qualify? This is scam by misleading people in believing they will would qualify so they will open an account with them. See the advertised deal here, you can see nothing about the states. \n

### Building the model

In [37]:
embedding = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1" #pretrained embedding from tfhub trained on google news of dimension 128
hub_layer = hub.KerasLayer(embedding, output_shape=[128], input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_data[:1])

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[ 0.33617228,  0.23203762, -0.09538511, -0.06548779, -0.08765431,
        -0.01077191, -0.00245969,  0.05258208, -0.06241624,  0.19836168,
         0.09080565, -0.12921022,  0.0396726 , -0.05281023,  0.00817845,
         0.01611447, -0.17562987,  0.07727271, -0.1941034 ,  0.28781003,
        -0.18966144,  0.22917113,  0.02318892, -0.05648421,  0.0852662 ,
        -0.15905663,  0.12677582, -0.08696102, -0.10879584,  0.04013509,
         0.04865107, -0.06219418,  0.01057932, -0.09699482, -0.10901407,
         0.12662937, -0.00856555, -0.17180865, -0.16313194,  0.02499727,
         0.05349051, -0.18023016,  0.03826252,  0.04737705,  0.06879856,
         0.07156484, -0.10703883,  0.02409267, -0.02078928,  0.06535411,
        -0.05628244,  0.12537208,  0.01271748, -0.00653138,  0.02049752,
        -0.0382164 , -0.05212896, -0.0351408 , -0.11024247,  0.04053358,
        -0.07984848,  0.0055955 , -0.07771305, -0.0565149 ,  0.10259036,
 

In [38]:
train_data[:1]

<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'I am a victim of identity theft and this debt does not belong to me. \n\nPlease see the identity theft report and legal affidavit attached.'],
      dtype=object)>

In [39]:
model = tf.keras.Sequential()
model.add(hub_layer)
for units in [128, 128, 64 , 32]:                #for loop for testing out different models for easy editing
    model.add(tf.keras.layers.Dense(units, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(6, activation='softmax'))     #6 class softmax

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 128)               124642688 
                                                                 
 dense_10 (Dense)            (None, 128)               16512     
                                                                 
 dropout_8 (Dropout)         (None, 128)               0         
                                                                 
 dense_11 (Dense)            (None, 128)               16512     
                                                                 
 dropout_9 (Dropout)         (None, 128)               0         
                                                                 
 dense_12 (Dense)            (None, 64)                8256      
                                                                 
 dropout_10 (Dropout)        (None, 64)               

In [40]:
def weightedLoss(originalLossFunc, weightsList):

    def lossFunc(true, pred):

        axis = -1 #if channels last 
        #axis=  1 #if channels first


        #argmax returns the index of the element with the greatest value
        #done in the class axis, it returns the class index    
        classSelectors = K.argmax(true, axis=axis) 
            #if your loss is sparse, use only true as classSelectors

        #considering weights are ordered by class, for each class
        #true(1) if the class index is equal to the weight index   
        classSelectors = [K.equal(i, classSelectors) for i in range(len(weightsList))]

        #casting boolean to float for calculations  
        #each tensor in the list contains 1 where ground true class is equal to its index 
        #if you sum all these, you will get a tensor full of ones. 
        classSelectors = [K.cast(x, K.floatx()) for x in classSelectors]

        #for each of the selections above, multiply their respective weight
        weights = [sel * w for sel,w in zip(classSelectors, weightsList)] 

        #sums all the selections
        #result is a tensor with the respective weight for each element in predictions
        weightMultiplier = weights[0]
        for i in range(1, len(weights)):
            weightMultiplier = weightMultiplier + weights[i]


        #make sure your originalLossFunc only collapses the class axis
        #you need the other axes intact to multiply the weights tensor
        loss = originalLossFunc(true,pred) 
        loss = loss * weightMultiplier

        return loss
    return lossFunc

In [41]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [42]:
train_data_f=train_data_f.shuffle(70000).batch(512)    #creating batched of the training data and shuffling the data
test_data_f=test_data_f.batch(512)

In [43]:
history = model.fit(train_data_f,
                    epochs=10,
                    validation_data=test_data_f,
                    verbose=1,
                    class_weight=weights)     #class weights would be used for the calcualtion of the losses

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


90/90 [==============================] - 16s 140ms/step - loss: 1.7898 - accuracy: 0.1872 - val_loss: 1.7429 - val_accuracy: 0.4599
Epoch 2/10
90/90 [==============================] - 15s 136ms/step - loss: 1.4713 - accuracy: 0.4567 - val_loss: 0.8099 - val_accuracy: 0.7699
Epoch 3/10
90/90 [==============================] - 16s 140ms/step - loss: 0.9763 - accuracy: 0.7331 - val_loss: 0.5780 - val_accuracy: 0.8235
Epoch 4/10
90/90 [==============================] - 16s 137ms/step - loss: 0.7035 - accuracy: 0.8220 - val_loss: 0.5200 - val_accuracy: 0.8615
Epoch 5/10
90/90 [==============================] - 15s 137ms/step - loss: 0.5220 - accuracy: 0.8677 - val_loss: 0.5220 - val_accuracy: 0.8630
Epoch 6/10
90/90 [==============================] - 15s 137ms/step - loss: 0.4027 - accuracy: 0.8933 - val_loss: 0.5151 - val_accuracy: 0.8716
Epoch 7/10
90/90 [==============================] - 16s 137ms/step - loss: 0.3332 - accuracy: 0.9111 - val_loss: 0.5241 - val_accuracy: 0.8763
Epoch 8/10

In [44]:
len(list(dataset_test))

11491

In [45]:
results = model.evaluate(dataset_test.map(fetch).batch(11491), verbose=2)

print(results)
#a better metric for evaluation would be precision & recall as we have an imbalanced dataset here

1/1 - 2s - loss: 0.6582 - accuracy: 0.8628 - 2s/epoch - 2s/step
[0.6581988334655762, 0.8628491759300232]


In [47]:
test_data, test_labels = next(iter(dataset_test.map(fetch).batch(20000)))

In [48]:
y_pred=model.predict(test_data)

In [49]:
from sklearn.metrics import classification_report
print(classification_report(test_labels.numpy().argmax(axis=1), y_pred.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.96      0.85      0.90      4345
           1       0.85      0.83      0.84      2633
           2       0.89      0.95      0.92      2016
           3       0.79      0.88      0.83      1384
           4       0.82      0.82      0.82       576
           5       0.54      0.81      0.65       537

    accuracy                           0.86     11491
   macro avg       0.81      0.86      0.83     11491
weighted avg       0.88      0.86      0.87     11491



In [50]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_labels.numpy().argmax(axis=1), y_pred.argmax(axis=1))

array([[3698,  200,  122,   72,   77,  176],
       [  82, 2187,   37,  214,   12,  101],
       [  20,   23, 1910,   20,    5,   38],
       [  11,  114,   21, 1211,    4,   23],
       [  27,   11,   27,    4,  472,   35],
       [  26,   30,   29,    8,    7,  437]])

### Training using LSTM

In [79]:
model1 = tf.keras.Sequential()
model1.add(hub_layer)
model1.add(tf.keras.layers.Reshape( target_shape=(128 , 1 ) ))
model1.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)))
model1.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)))
for units in [128, 64]:
  model1.add(tf.keras.layers.Dense(units, activation='relu'))
  # model1.add(tf.keras.layers.BatchNormalization())
  model1.add(tf.keras.layers.Dropout(0.5))
model1.add(tf.keras.layers.Dense(6, activation='softmax'))

model1.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 128)               124642688 
                                                                 
 reshape_4 (Reshape)         (None, 128, 1)            0         
                                                                 
 bidirectional_8 (Bidirectio  (None, 128, 128)         33792     
 nal)                                                            
                                                                 
 bidirectional_9 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense_27 (Dense)            (None, 128)               8320      
                                                                 
 dropout_18 (Dropout)        (None, 128)              

In [80]:
model1.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [81]:
train_data_f=dataset_train.map(fetch)
test_data_f=dataset_test.map(fetch)

In [82]:
train_data_f=train_data_f.shuffle(70000).batch(512)
test_data_f=test_data_f.batch(512)

In [83]:
history = model1.fit(train_data_f,
                    epochs=10,
                    validation_data=test_data_f,
                    verbose=1,
                    class_weight=weights)

Epoch 1/10
90/90 [==============================] - 33s 262ms/step - loss: 0.6715 - accuracy: 0.7613 - val_loss: 0.7736 - val_accuracy: 0.8338
Epoch 2/10
90/90 [==============================] - 25s 242ms/step - loss: 0.0817 - accuracy: 0.9763 - val_loss: 0.8901 - val_accuracy: 0.8396
Epoch 3/10
90/90 [==============================] - 24s 243ms/step - loss: 0.0527 - accuracy: 0.9844 - val_loss: 1.0726 - val_accuracy: 0.8358
Epoch 4/10
90/90 [==============================] - 25s 241ms/step - loss: 0.0500 - accuracy: 0.9846 - val_loss: 1.0734 - val_accuracy: 0.8310
Epoch 5/10
90/90 [==============================] - 24s 242ms/step - loss: 0.0454 - accuracy: 0.9859 - val_loss: 1.1296 - val_accuracy: 0.8366
Epoch 6/10
90/90 [==============================] - 25s 242ms/step - loss: 0.0364 - accuracy: 0.9893 - val_loss: 1.1899 - val_accuracy: 0.8394
Epoch 7/10
90/90 [==============================] - 24s 243ms/step - loss: 0.0309 - accuracy: 0.9909 - val_loss: 1.3367 - val_accuracy: 0.8394

In [65]:
test_data, test_labels = next(iter(dataset_test.map(fetch).batch(20000)))
y_pred=model.predict(test_data)

In [66]:
from sklearn.metrics import classification_report
print(classification_report(test_labels.numpy().argmax(axis=1), y_pred.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.92      0.91      0.91      4345
           1       0.86      0.84      0.85      2633
           2       0.94      0.90      0.92      2016
           3       0.81      0.87      0.84      1384
           4       0.76      0.83      0.79       576
           5       0.66      0.72      0.69       537

    accuracy                           0.87     11491
   macro avg       0.82      0.84      0.83     11491
weighted avg       0.88      0.87      0.87     11491



In [67]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_labels.numpy().argmax(axis=1), y_pred.argmax(axis=1))

array([[3943,  140,   58,   58,   81,   65],
       [ 153, 2206,   17,  187,   16,   54],
       [  65,   36, 1810,   28,   34,   43],
       [  29,  126,   13, 1202,    6,    8],
       [  45,   12,    8,    2,  478,   31],
       [  63,   48,   11,   13,   17,  385]])

In [109]:
i = 88
test_data[i].numpy()

b'Numerous call received and message left regarding debt. Called back- debt in relation to family member, whom I do not speak with ( last I heard he was in jail ). Requested I not be contacted regarding this debt any longer. \n\nCalls continued- including leaving numerous voicemails/ day.'

In [110]:
y_pred[i].argmax()

0

In [111]:
test_labels[i].numpy().argmax()

0